<a href="https://colab.research.google.com/github/VellummyilumVinoth/Aspect_based_sentimental_analysis/blob/main/Testing_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [2]:
# Connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install demoji
! pip install transformers
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 5.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import demoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [62]:
# Take input as a single sentence
input_text = input("Enter a sentence: ") # The price is good but quality is bad and design is neutral

Enter a sentence: The price is good but quality is bad and design is neutral


In [63]:
print("Input Sentence: ", input_text)

Input Sentence:  The price is good but quality is bad and design is neutral


In [64]:
# Download the demoji library's emoji descriptions (only required once)
demoji.download_codes()

# Define a function to replace emojis with their corresponding descriptions
def replace_emojis(text):
    emojis = demoji.findall(text)

    for emoji in emojis:
        text = text.replace(emoji, " " + emojis[emoji].split(":")[0])

    return text

# Apply the function to the input sentence
sentence_without_emoji = replace_emojis(input_text)

<ipython-input-64-a1616bdffc47>:2: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [65]:
print("Sentence without emojis:", sentence_without_emoji)

Sentence without emojis: The price is good but quality is bad and design is neutral


In [66]:
import re

# Download NLTK resources (only required once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define a function to remove symbols
def remove_symbols(text):
    # Lowercase the text
    text = text.lower()

    # Define the pattern to match the symbols you want to remove
    pattern = r'[-,"\'’\.!:;…_+=@#$%^&*{\[()\]}]'

    # Remove the symbols using regular expressions
    text = re.sub(pattern, ' ', text)

    return text

# Apply the symbol removal function to the sentence without emoji
sentence_without_symbols = remove_symbols(sentence_without_emoji)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [67]:
print("Sentence without symbols:", sentence_without_symbols)

Sentence without symbols: the price is good but quality is bad and design is neutral


In [68]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to preprocess the text
def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)

    return preprocessed_text

# Apply the preprocessing function to the sentence without symbols
preprocessed_sentence = preprocess_text(sentence_without_symbols)


In [69]:
print("Preprocessed Sentence:", preprocessed_sentence)

Preprocessed Sentence: price good quality bad design neutral


In [70]:
import nltk
import torch
import pandas as pd
import spacy
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, BertTokenizer

nltk.download("punkt")

tokenizer = T5Tokenizer.from_pretrained("flax-community/t5-base-wikisplit")
model = T5ForConditionalGeneration.from_pretrained("flax-community/t5-base-wikisplit")

def add_full_stop(text):
    last_character = text[-1]
    if last_character != ".":
        text += ""
    return text

input_sentence = add_full_stop(sentence_without_symbols)

encoder_max_length = 256
decoder_max_length = 256

complex_tokenized = tokenizer(input_sentence,
                              padding="max_length",
                              truncation=True,
                              max_length=encoder_max_length,
                              return_tensors='pt')

simple_tokenized = model.generate(complex_tokenized['input_ids'], attention_mask=complex_tokenized['attention_mask'], max_length=decoder_max_length, num_beams=5)
simple_sentences = tokenizer.batch_decode(simple_tokenized, skip_special_tokens=True, clean_up_tokenization_spaces=True)

nlp = spacy.load("en_core_web_sm")

conjunctive_words = ['and', 'but', 'or', 'so', 'yet', 'for', 'nor']

if len(simple_sentences) > 0:
    input_sentence = simple_sentences[0]
    sentences = list(nlp(input_sentence).sents)
    output_sentences = []
    for sentence in sentences:
        sub_sentences = [sentence.text]
        for conjunctive_word in conjunctive_words:
            temp_sentences = []
            for sub_sentence in sub_sentences:
                temp_sentences.extend(sub_sentence.split(conjunctive_word))
            sub_sentences = temp_sentences
        for sub_sentence in sub_sentences:
            output_sentence = add_full_stop(sub_sentence)
            output_sentences.append(output_sentence)

    input_statements = [statement.strip() for statement in output_sentences if statement.strip()]
    print(input_statements)
else:
    input_sentence = ""


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['The price is good', 'quality is bad', 'design is neutral.']


In [71]:
class SentimentAnalysisModel:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def predict_sentiment(self, aspect_sentence):
        inputs = self.tokenizer.encode_plus(aspect_sentence, add_special_tokens=True, return_tensors="pt")
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        probabilities = torch.softmax(logits, dim=1)
        sentiment_score = probabilities[0][1].item()  # Positive sentiment score
        return sentiment_score

    def assign_sentiment_label(self, sentiment_score):
        if sentiment_score >= 0.99983:
            sentiment_label = 'very good'
        elif sentiment_score >= 0.999:
            sentiment_label = 'good'
        elif sentiment_score > 0.001:
            sentiment_label = 'neutral'
        elif sentiment_score > 0.0002:
            sentiment_label = 'bad'
        else:
            sentiment_label = 'poor'
        return sentiment_label

# Load tokenizer and model for aspect sentiment analysis
aspect_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
aspect_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Load tokenizer and model for aspect classification
classification_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/zidan/finetuned_bert', num_labels=7)
classification_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/zidan/finetuned_bert', do_lower_case=True)

# Create sentiment analysis model instance for aspect sentiment analysis
sentiment_analysis_model = SentimentAnalysisModel(aspect_tokenizer, aspect_model)

# Define aspect labels
aspect_labels = ["price", "quality", "delivery cost", "packaging", "design", "customer service", "usability"]

# Step 1: Classify the aspect
encoded_inputs = classification_tokenizer.batch_encode_plus(
    input_statements,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=512
)

with torch.no_grad():
    classification_model.eval()
    outputs = classification_model(**encoded_inputs)
    predicted_labels = outputs.logits

predicted_labels = (torch.sigmoid(predicted_labels) > 0.5).to(torch.int).tolist()

# Create DataFrame with input statements, aspect labels, and predicted labels
df_data = {"Statement": input_statements}
for i, aspect_label in enumerate(aspect_labels):
    df_data[aspect_label] = [label[i] for label in predicted_labels]

df = pd.DataFrame(df_data)

df

,Statement,price,quality,delivery cost,packaging,design,customer service,usability
0,The price is good,1,0,0,0,0,0,0
1,quality is bad,0,1,0,0,0,0,0
2,design is neutral.,0,0,0,0,1,0,0


In [72]:
# Step 2: Perform sentiment analysis for each aspect
aspect_sentiment_labels = {}
aspect_sentiment_scores = {}

for aspect_label in aspect_labels:
    aspect_sentiment_labels[aspect_label] = []
    aspect_sentiment_scores[aspect_label] = []

for statement_index, statement in enumerate(input_statements):
    for aspect_index, aspect_label in enumerate(aspect_labels):
        if predicted_labels[statement_index][aspect_index] == 1:
            sentiment_score = sentiment_analysis_model.predict_sentiment(statement)
            sentiment_label = sentiment_analysis_model.assign_sentiment_label(sentiment_score)
            aspect_sentiment_labels[aspect_label].append(sentiment_label)
            aspect_sentiment_scores[aspect_label].append(sentiment_score)
        else:
            aspect_sentiment_labels[aspect_label].append("-")
            aspect_sentiment_scores[aspect_label].append("-")

# Calculate the average sentiment score for each aspect
overall_sentiment_scores = []
for aspect_label in aspect_labels:
    numeric_scores = [score for score in aspect_sentiment_scores[aspect_label] if isinstance(score, float)]
    if numeric_scores:
        aspect_sentiment_scores[aspect_label] = sum(numeric_scores) / len(numeric_scores)
    else:
        aspect_sentiment_scores[aspect_label] = "-"

# Calculate the overall sentiment label and score for the input sentence
numeric_scores = [score for score in aspect_sentiment_scores.values() if isinstance(score, float)]
if numeric_scores:
    overall_sentiment_score = sum(numeric_scores) / len(numeric_scores)
    overall_sentiment_label = sentiment_analysis_model.assign_sentiment_label(overall_sentiment_score)
else:
    overall_sentiment_score = "-"
    overall_sentiment_label = "-"

# Create DataFrame from the result data
result_data = {
    "Review": input_sentence,
    "Overall Sentiment Score": overall_sentiment_score,
    "Overall Sentiment Label": overall_sentiment_label,
}
# Calculate the average sentiment label for each aspect
average_sentiment_labels = {}
for aspect_label in aspect_labels:
    sentiment_labels = [label for label in aspect_sentiment_labels[aspect_label] if label != "-"]
    if sentiment_labels:
        average_sentiment_labels[aspect_label] = max(set(sentiment_labels), key=sentiment_labels.count)
    else:
        average_sentiment_labels[aspect_label] = "-"

# Create a dictionary to store the result data
result_data = {
    "Review": [input_text],
    "price": [average_sentiment_labels["price"]],
    "quality": [average_sentiment_labels["quality"]],
    "delivery cost": [average_sentiment_labels["delivery cost"]],
    "packaging": [average_sentiment_labels["packaging"]],
    "design": [average_sentiment_labels["design"]],
    "customer service": [average_sentiment_labels["customer service"]],
    "usability": [average_sentiment_labels["usability"]],
    "Overall Sentiment Score": [overall_sentiment_score],
    "Overall Sentiment Label": [overall_sentiment_label],
}

# Create DataFrame from the result data with the desired column order
result_df = pd.DataFrame(result_data, columns=["Review", "price", "quality", "delivery cost","packaging",
                                               "design", "customer service", "usability",
                                               "Overall Sentiment Score", "Overall Sentiment Label"])

# Print the result
result_df

,Review,price,quality,delivery cost,packaging,design,customer service,usability,Overall Sentiment Score,Overall Sentiment Label
0,The price is good but quality is bad and desig...,good,bad,-,-,neutral,-,-,0.335306,neutral
